In [1]:
from pathlib import Path
import sys  
import os

In [4]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "libs")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))

In [5]:
sys.path

['/opt/miniconda3/envs/RecSys/lib/python312.zip',
 '/opt/miniconda3/envs/RecSys/lib/python3.12',
 '/opt/miniconda3/envs/RecSys/lib/python3.12/lib-dynload',
 '',
 '/opt/miniconda3/envs/RecSys/lib/python3.12/site-packages',
 '/opt/miniconda3/envs/RecSys/lib/python3.12/site-packages/setuptools/_vendor',
 '/Users/safuan/Python/RecSysCompetition2024Polimi/libs',
 '/Users/safuan/Python/RecSysCompetition2024Polimi/libs',
 '/Users/safuan/Python/RecSysCompetition2024Polimi']

In [5]:
from libs.Recommenders.Similarity.Compute_Similarity import SimilarityFunction
from libs.Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from libs.Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from libs.Recommenders.KNN.ItemKNNCBFCFSimilarityHybridRecommender import ItemKNNCBFCFSimilarityHybridRecommender
from libs.Evaluation.Evaluator import EvaluatorHoldout
from Utils.load_URM import load_URM
from Utils.load_ICM import load_ICM
from operator import itemgetter
import numpy as np

from src.utils import tf_idf, train_model, write_submission

from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_all = load_URM("../data/data_train.csv")
ICM_all = load_ICM("../data/data_ICM_metadata.csv")


In [7]:
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 167 ( 0.5%) Users that have less than 1 test interactions


In [16]:
MAP_CBF=[]
MAP_CF=[]

for similarity in ['cosine','pearson','jaccard','tanimoto','adjusted']:

    for topK in [10, 50, 100, 200, 500]:
        for shrink in [0, 10, 50, 100, 200, 500]:
            recommender = ItemKNNCFRecommender(URM_train)
            recommender.fit(topK=topK, shrink=shrink, similarity=similarity, normalize=False)

            result_dict, _ = evaluator_validation.evaluateRecommender(recommender)
            MAP_CF.append(('similarity={}'.format(similarity),'topK={}'.format(topK),'shrink={}'.format(shrink),result_dict["MAP"][10]))

            print(MAP_CF[-1])

    for topK in [10, 50, 100, 200, 500]:
        for shrink in [0, 10, 50, 100, 200, 500]:
            recommender = ItemKNNCBFRecommender(URM_train=URM_train, ICM_train=ICM_all)
            recommender.fit(topK=topK, shrink=shrink, similarity=similarity, normalize=False)

            result_dict, _ = evaluator_validation.evaluateRecommender(recommender)

            MAP_CBF.append(('similarity={}'.format(similarity),'topK={}'.format(topK),'shrink={}'.format(shrink),result_dict["MAP"][10]))
            print(MAP_CBF[-1])

Similarity column 38121 (100.0%), 5659.13 column/sec. Elapsed time 6.74 sec
EvaluatorHoldout: Processed 35556 (100.0%) in 11.91 sec. Users per second: 2985
('similarity=cosine', 'topK=10', 'shrink=0', 0.03414625397539596)
Similarity column 38121 (100.0%), 4877.37 column/sec. Elapsed time 7.82 sec
EvaluatorHoldout: Processed 35556 (100.0%) in 12.09 sec. Users per second: 2940
('similarity=cosine', 'topK=10', 'shrink=10', 0.03414292254382453)
Similarity column 38121 (100.0%), 4598.30 column/sec. Elapsed time 8.29 sec
EvaluatorHoldout: Processed 35556 (100.0%) in 12.41 sec. Users per second: 2865
('similarity=cosine', 'topK=10', 'shrink=50', 0.03414281317019169)
Similarity column 38121 (100.0%), 4883.94 column/sec. Elapsed time 7.81 sec
EvaluatorHoldout: Processed 35556 (100.0%) in 11.87 sec. Users per second: 2996
('similarity=cosine', 'topK=10', 'shrink=100', 0.03414281317019169)
Similarity column 38121 (100.0%), 4722.85 column/sec. Elapsed time 8.07 sec
EvaluatorHoldout: Processed 3555

In [17]:
max_tuple = max(MAP_CF, key=itemgetter(3))
print('The best combination for the ICF recommender is ({},{},{}), with a MAP = {}'.format(max_tuple[0],max_tuple[1], max_tuple[2], max_tuple[3]))

max_tuple = max(MAP_CBF, key=itemgetter(3))
print('The best combination for the CBF recommender is ({},{},{}), with a MAP = {}'.format(max_tuple[0],max_tuple[1], max_tuple[2], max_tuple[3]))


The best combination for the ICF recommender is (similarity=jaccard,topK=10,shrink=10), with a MAP = 0.050442041349481284
The best combination for the CBF recommender is (similarity=jaccard,topK=10,shrink=50), with a MAP = 0.0149851419267978


In [9]:
recommender = ItemKNNCFRecommender(URM_train)
recommender.fit(topK=10, shrink=10, similarity='jaccard', normalize=False)

result_dict, _ = evaluator_validation.evaluateRecommender(recommender)


Similarity column 38121 (100.0%), 5006.58 column/sec. Elapsed time 7.61 sec
EvaluatorHoldout: Processed 35569 (100.0%) in 8.90 sec. Users per second: 3998


In [2]:
import pandas as pd

In [19]:
result_dict["MAP"][10]

0.05107207901341492

In [26]:
recommender.recommend(1000, cutoff=10)

([6842, 6374, 2, 6853, 2836, 21442, 6852, 6858, 19208, 6611],
 array([[0.        , 0.        , 0.44027978, ..., 0.        , 0.        ,
         0.        ]], dtype=float32))

In [32]:
def write_submission_lib_model(trained_model, filename: str = "submission.csv") -> None:
	"""Builds the submission file from a trained recommender model. The file is saved in a CSV format.

	:param trained_model: A fitted recommender model
	:type trained_model: RecommenderModel
	:param filename: The filename of the submission for this particular recommender model
	:type filename: str
	"""
	target_users_test = pd.read_csv("../data/data_target_users_test.csv",).to_numpy().ravel()

	recommendations = np.array([
		trained_model.recommend(user_id, cutoff=10) for user_id in target_users_test
	])

	if not os.path.exists("../submissions"):
		os.makedirs("../submissions")
	with open(f"../submissions/{filename}", "w") as f:
		f.write("user_id,item_list\n")
		for user_id, recs in zip(target_users_test, recommendations):
			f.write(f"{user_id},{' '.join(map(str, recs))}\n")

In [35]:
write_submission_lib_model(recommender, filename='cbf_urm_lib.csv')